</br>

# 建物内温湿度環境解析ネットワークモデル  


※2021/01/12 加筆・修正  
初めて本プログラムを使う際は以下のセルをMarkdownからCodeに変更し、CSVおよびDataFramesパッケージをインストールすること

using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("Dates")
Pkg.add("XLSX")
Pkg.add("StringEncodings")

## 手順0：計算に必要なモジュールを読み込む  

In [1]:
using Dates # 解析内時間を計算するためのモジュール
include("./module/building_network_model.jl");  # 空間の構成要素を表すモジュール
include("./module/transfer_in_media.jl"); # 材料中における熱・水分の移動を表すモジュール
include("./logger.jl"); # 計算結果を保存するモジュール

## 手順１：解析対象モデルの構築

解析対象モデルの概念図を示す。  
本モデルでは建物内の温湿度環境が以下の4つの要素により構成されるものと考える。  
- 室空気
- 壁体
- 隙間・開口
- 外界気象

なお、各要素の与え方は./input_data/building_network_model内部およびclimate_dataのcsvファイルで定義する。

<img src = "./input_data/picture/BNMモデル図.png" width="50%">

#### csvファイルに関する留意点2024/12/04  
##### ①room condition  
- 特になし  

##### ②wall condition  
- 上流側と下流側の関係について：必ず数字の小さい方の部屋を上流側に配置すること  
- 第一種境界条件の場合IPに0を、第二種境界条件の場合IPに-1を入力し、流量条件をcsvファイルに記入すること
- 材料名称は./module/material_propertyフォルダ内の材料名と同一のものを使用すること  
- 壁の向きIONは傾斜角を、壁面方位は方位角を表す。なお、壁面方位は境界面が面している方角を入力すること。（北面側の面ならNを入力）  
- 日射の計算はインシデンス行列を用いて（隣接面がclimateであるかどうかから）自動で判別され計算される。
- 湿気伝達率は空欄の場合対流熱伝達率を用いてLewis関係（ルイス数=1）として計算される。  
- 家具や収蔵物などによる熱容量・湿気容量を考慮する場合、IP側を-1とし、熱流・水分流を0とした片端を断熱・断湿条件とした物質を与え、適切な表面積を与えることで計算を行うことが有用である。

##### ③opening condition  
- 計算タイプに応じた情報がすべて含まれているため、すべての値を入力する必要はない。計算内容に応じて適宜入力内容を決めること。  
- constant（換気回数あるいは換気量）で計算する場合、必要な情報はQupとQdwのみである。  

##### ④climate
- climate data内のデータは不要な値に関しては空欄で問題ない。気象庁から得られるデータをすべて入力できるようにしてある。  
- 現時点では全天日射量を入力しても計算を行うことはできない。別途、大気透過率を計算したのち入力すること。　　
- 環境データは./moduleフォルダ内のclimateにて処理を行っている。基本的には前後の時刻の値を直線内挿することで計算を行っている。  
- climate内に時刻を表すdateが含まれており、これは解析内における計算時間と一致するようになっている。

In [2]:
network_model = create_BNM_model(
    file_name_rooms     = "./input_data/building_network_model/room_condition.csv", 
    file_name_walls     = "./input_data/building_network_model/wall_condition.csv", 
    file_name_openings  = "./input_data/building_network_model/opening_condition.csv",
    file_name_climate   = "./input_data/climate_data/climate_data_nagoya.csv")

 壁番号 = 25, IP = 0
は第一種あるいは第二種境界条件です。
 壁番号 = 26, IP = -1
は第一種あるいは第二種境界条件です。


BNM(Room[Room(1, "outdoor", Air(0, "climate", 0.0, 0.0, 0.0, 0.0, 282.54999999999995, 0.8900000000000001, 1049.7515514101506, 0.006511531759623337, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, "OFF"), Room(2, "storage room", Air(0, "storage room", 0.0, 0.0, 0.0, 646.2, 283.15, 0.56, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, "OFF"), Room(3, "special storage room", Air(0, "special storage room", 0.0, 0.0, 0.0, 112.1, 283.15, 0.56, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, "OFF"), Room(4, "machine room", Air(0, "machine room", 0.0, 0.0, 0.0, 64.1, 283.15, 0.56, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.025, 0.0, 0.0, 0.0, "OFF"), Room(5, "vestibule", Air(0, "vestibule", 0.0, 0.0, 0.0, 57.9, 283.15, 0.56, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, "OFF")

# 初期温度の再設
# 壁ごとの計算
for i = 1 : length(network_model.walls)
    # 各壁体内部の収支計算
    for j = 1 : length(network_model.walls[i].cell)
        network_model.walls[i].cell[j].temp = 30.0 + 273.15
        network_model.walls[i].cell[j].miu  = convertRH2Miu( temp = network_model.walls[i].cell[j].temp, rh = 0.6 )
    end    
end

# 空気ごとの計算
for i = 2 : length(network_model.rooms)   # 外界気象の計算を除外
    set_temp(network_model.rooms[i], 30.0 + 273.15)
    set_rh(network_model.rooms[i], 0.6)
end

In [3]:
# 確認作業
network_model.rooms
network_model.walls
network_model.openings

15-element Vector{Opening}:
 Opening(1, 1, 5, "constant", 0.016, 0.016, "VT", "N", "S", 0.998, 2.0, 0.0, 1.999, 0.005, 1.5, 1.5, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 0.0, 0.0, 1, Room(1, "outdoor", Air(0, "climate", 0.0, 0.0, 0.0, 0.0, 282.54999999999995, 0.8900000000000001, 1049.7515514101506, 0.006511531759623337, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, "OFF"), Room(5, "vestibule", Air(0, "vestibule", 0.0, 0.0, 0.0, 57.9, 283.15, 0.56, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, "OFF"), 0.0, Dict("DWU" => 0.0, "W" => 0.0, "WU" => 0.0, "DW" => 0.0, "DWD" => 0.0, "WD" => 0.0))
 Opening(2, 2, 5, "constant", 0.008, 0.008, "VT", "E", "W", 0.79, 2.0, 0.0, 1.995, 0.01, 1.8, 1.8, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 0.0, 0.0, 1, Room(2, "storage room", Air(0, "storage room", 0.0, 0.0, 0.0, 646.2, 283.15, 0.56, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 

## 手順２：流量計算式および収支式を構築する

### 手順 2.1：壁面日射量・降水量の計算

In [4]:
# 各壁面に流入する日射量・降水量の計算

function cal_boundary_condtion_of_BNM( network::BNM )
    for i = 1 : length(network.walls)
        if network.IC_walls[1,i] == 1   # インシデンス行列よりIP側がClimateである場合を判別
            BC = :BC_IP
        elseif network.IC_walls[1,i] == -1   # インシデンス行列よりIM側がClimateである場合を判別
            BC = :BC_IM
        else
            continue # 以下の動作をスキップ
        end
        # 値の初期化
        set_q_added(  getfield(network.walls[i], BC), 0.0)
        set_jv_added( getfield(network.walls[i], BC), 0.0)
        set_jl_added( getfield(network.walls[i], BC), getfield(getfield(network.walls[i], BC), :jl_surf)) # 表面水（結露水）の追加
        set_jl_surf(  getfield(network.walls[i], BC), 0.0)
        # 直達日射量
        add_q_added( getfield(network.walls[i], BC), ar(getfield(network.walls[i], BC)) * cal_direct_solar_radiation_at_wall(network.climate, getfield(network.walls[i], BC) ) )
        # 天空日射量
        add_q_added( getfield(network.walls[i], BC), ar(getfield(network.walls[i], BC)) * cal_diffused_solar_radiation_at_wall(network.climate, getfield(network.walls[i], BC) ) )
        # 地面反射日射
        add_q_added( getfield(network.walls[i], BC), ar(getfield(network.walls[i], BC)) * cal_reflected_solar_radiation_by_ground(network.climate, getfield(network.walls[i], BC) ) )
        # 夜間放射
        add_q_added( getfield(network.walls[i], BC), ar(getfield(network.walls[i], BC)) * cal_effective_thermal_radiation_at_wall(network.climate, getfield(network.walls[i], BC) ) )
        # 降水量の計算
        #add_jl_added(getfield(network.walls[i], BC), cal_effective_rainfall_at_wall( network.climate, getfield(network.walls[i], BC) ))
    end
end

cal_boundary_condtion_of_BNM (generic function with 1 method)

### 手順 2.2：熱・水分流の計算

In [5]:
function cal_network_flux_of_wall( network::BNM )

    # 室に加わる熱流量の初期化
    for i = 1 : length(network.rooms)
        set_H_wall(network.rooms[i], 0.0)
        set_J_wall(network.rooms[i], 0.0)
    end

    # 壁ごとの計算
    for i = 1 : length(network.walls)
        target_model = network.walls[i].target_model

        # 各壁体内部の流量計算
        for j = 1 : length(target_model)-1

            # 熱流・水分流の計算
            # 通気層内
            if  material_name(target_model[j+1]) == "vented_air_space"
                q  = 1.0 / Rthx(target_model[j+1]) * ( temp(target_model[j]) - temp(target_model[j+2]) )# + 放射熱伝達はすでに含まれている。
                        #5.67e-8 / ( 2/0.9 - 1 ) * ( temp(target_model[j])^4 - temp(target_model[j+2])^4 ) # 熱伝達および放射熱伝達※放射率0.9
                jv = 1.0 / Rdpx(target_model[j+1]) * ( pv(target_model[j])   - pv(target_model[j+2]) )
                jl = 0.0
            elseif material_name(target_model[j]) == "vented_air_space"
                q  = 1.0 / Rthx(target_model[j]) * ( temp(target_model[j-1]) - temp(target_model[j+1]) )# + 
                        #5.67e-8 / ( 2/0.9 - 1 ) * ( temp(target_model[j-1])^4 - temp(target_model[j+1])^4 ) # 熱伝達および放射熱伝達※放射率0.9
                jv = 1.0 / Rdpx(target_model[j]) * ( pv(target_model[j-1])   - pv(target_model[j+1]) )
                jl = 0.0
            # 材料内あるいは材料境界
            else
                q =  cal_q( target_model[j], target_model[j+1] )
                jv = cal_jv( target_model[j], target_model[j+1] ) 
                jl = cal_jl( target_model[j], target_model[j+1], sin(network.walls[i].ION / (180.0/pi)) ) # 傾斜角のsin成分
            end

            # cellおよびairに熱流・水分流を追加
            # 流出側
            if      typeof(target_model[j]) == Cell
                target_model[j].Q[2][1]  = - q  * dy(target_model[j]) * dz(target_model[j])
                target_model[j].Jv[2][1] = - jv * dy(target_model[j]) * dz(target_model[j])
                target_model[j].Jl[2][1] = - jl * dy(target_model[j]) * dz(target_model[j])
            elseif  typeof(target_model[j]) == BC_Robin
                add_H_wall(target_model[j].air, - q * area(network.walls[i]) )
                add_J_wall(target_model[j].air, - jv* area(network.walls[i]) )
            end
            # 流入側
            if      typeof(target_model[j+1]) == Cell
                target_model[j+1].Q[1][1]  = q  * dy(target_model[j+1]) * dz(target_model[j+1])
                target_model[j+1].Jv[1][1] = jv * dy(target_model[j+1]) * dz(target_model[j+1])
                target_model[j+1].Jl[1][1] = jl * dy(target_model[j+1]) * dz(target_model[j+1])
            elseif  typeof(target_model[j+1]) == BC_Robin
                add_H_wall(target_model[j+1].air, q * area(network.walls[i]) )
                add_J_wall(target_model[j+1].air, jv* area(network.walls[i]) )
            end
        end

    end
    
    return
end

cal_network_flux_of_wall (generic function with 1 method)

In [6]:
# 確認
cal_network_flux_of_wall( network_model )

### 手順 2.3：換気による熱・水分流量の計算  

2024/11/15：将来的に換気回路網計算に変更予定

In [7]:
function cal_network_flux_of_ventilation( network::BNM )

    # 枝ごとの換気量の初期化
    for i = 1 : length(network.rooms)
        set_H_vent(network.rooms[i], 0.0)
        set_J_vent(network.rooms[i], 0.0)
    end

    for i = 1 : length(network.openings)
        if network.openings[i].Type == "constant"
            # 空気の比熱容量：乾き空気＋水蒸気の比熱
            ca_IP   = 1005.0 + 1846.0 * ah(room_IP(network.openings[i]))
            ca_IM   = 1005.0 + 1846.0 * ah(room_IM(network.openings[i]))
            # 空気の密度：ボイルシャルルの法則より
            rho_IP  =  353.25 / temp(room_IP(network.openings[i]))
            rho_IM  =  353.25 / temp(room_IM(network.openings[i]))
            rho     =  ( rho_IP + rho_IM ) / 2.0
            
            # 上流側室に加わる熱・水分量：風上法
            add_H_vent( room_IP(network.openings[i]), Qup(network_model.openings[i]) * rho * ( ca_IM  * temp(room_IM(network.openings[i])) - ca_IP * temp(room_IP(network.openings[i]) ) ) )
            add_J_vent( room_IP(network.openings[i]), Qup(network_model.openings[i]) * rho * ( ah(room_IM(network.openings[i])) - ah(room_IP(network.openings[i])) ) )
            # 下流側室に加わる熱・水分量：風上法
            add_H_vent( room_IM(network.openings[i]), Qdw(network_model.openings[i]) * rho * ( ca_IP * temp(room_IP(network.openings[i])) - ca_IM  * temp(room_IM(network.openings[i])) ) )
            add_J_vent( room_IM(network.openings[i]), Qdw(network_model.openings[i]) * rho * ( ah(room_IP(network.openings[i])) - ah(room_IM(network.openings[i])) ) )
            
        end
    end
    return
end

cal_network_flux_of_ventilation (generic function with 1 method)

In [8]:
# 確認
cal_network_flux_of_ventilation( network_model )

### 手順2.4：熱・水分収支の計算

In [9]:
function cal_new_value_ver_network( network::BNM, dt )
    
    # 壁ごとの計算
    for i = 1 : length(network.walls)
        target_model = network.walls[i].cell

        # 各壁体内部の収支計算
        for j = 1 : length(target_model)
            if  material_name(target_model[j]) ≠ "vented_air_space"
                # 熱収支計算
                target_model[j].temp = cal_newtemp( target_model[j], sum(sum(target_model[j].Q)), 
                    - sum(sum(target_model[j].Jv)), dt )

                # 水分収支計算
                nmiu = cal_newmiu(  target_model[j], sum(sum(target_model[j].Jv)), 
                    sum(sum(target_model[j].Jl)), dt )
                
                # 表面水・結露水の処理：雨水浸透などにより含水率が飽和度を超える場合、境界条件に蓄積水分量を保持させる
                # 未検証2024/12/05
                if j == 1 || j == length(target_model)
                    if j == 1
                        BC = :BC_IP
                    else j == length(target_model)
                        BC = :BC_IM
                    end
                    if nmiu >= - 0.1
                        nmiu    = - 0.1
                        # println( network.climate.date, " 壁体番号[", i, "]の表面にて結露が発生しました。" )
                        # 水分科学ポテンシャルが-0.1を超えた分は境界条件BCのjl_surfに蓄えられる。
                        setfield!( getfield(network.walls[i], BC), :jl_surf, 
                                    sum(sum(target_model[j].Jv)) + sum(sum(target_model[j].Jl)) ) # -                                   # cell内に流入する正味の水分量
                                    #dx(target_model[j]) * (phi(target_model[j]) - phi(setfield!(target_model[j], :miu, - 0.1)) ) )  # 飽和（miu=-0.1）との差分
                    else
                    end
                # 内部結露の処理
                else
                    if nmiu >= - 0.1
                        nmiu    = - 0.1
                        # 例外処理のためエラーを表示
                        println( network.climate.date, " 壁体番号[", i, "]の内部のcell[", j,  "]にて結露が発生しました。" )
                    end
                end

                target_model[j].miu = nmiu

            # 空気層内は前後の温度・水分状態の平均とする：※後ろのcellは値を更新できていないことに注意
            elseif  material_name(target_model[j]) == "vented_air_space"
                target_model[j].temp = ( temp(target_model[j-1]) + temp(target_model[j+1]) ) / 2.0
                target_model[j].miu  = ( miu(target_model[j-1])  + miu(target_model[j+1])  ) / 2.0
            end
        end    
    end
    
    # 空気ごとの計算
    for i = 2 : length(network.rooms)   # 外界気象の計算を除外
        set_temp(network.rooms[i], cal_newtemp( network.rooms[i].air, dt ))
        set_rh(network.rooms[i],  cal_newRH(   network.rooms[i].air, dt ))
    end
end

cal_new_value_ver_network (generic function with 1 method)

In [10]:
# 確認
cal_new_value_ver_network( network_model, 1.0 )

## 手順３：計算条件に関する設定  


計算条件として、下記の設定を行う。
- dt : 計算の時間刻み 
- end_date : 計算終了時刻  

なお、計算の開始時刻は気象データ（climate_data）の始まりからとなっている。  
終了時刻の設定方法は
もし計算の開始時刻を変更する場合、BNM（Building Network Model）内のclimate内のdateを変更すること。

なお、dtは実数（小数点以下も可）、dateはJuliaのDateTimeを使って設定すること。

In [ ]:
# 計算の時間刻み
dt = 1.0

# 計算開始時刻
network_model.climate.date = DateTime(2022,4,1,00,00,00)

# 計算終了時刻
end_date = network_model.climate.date + Month(18)
#end_date = DateTime(2012,4,8,00,00,00)

reset_climate_data(network_model.climate)

0.8900000000000001

また、日射量の計算のため、解析の対象となる都市の緯度、経緯情報を入力する。

In [12]:
#= network_model.climate.location["city"]  = "Kyoto"   # 都市名
network_model.climate.location["lon"]   = 135.678   # 緯度
network_model.climate.location["phi"]   = 34.983    # 経度
network_model.climate.location["lons"]  = 135.0     # 地方標準時の地点の経度
 =#

network_model.climate.location["city"]  = "Nagoya"   # 都市名
network_model.climate.location["lon"]   = 136.906   # 緯度
network_model.climate.location["phi"]   = 35.181    # 経度
network_model.climate.location["lons"]  = 135.0     # 地方標準時の地点の経度

135.0

## 手順４：計算結果の保存

計算結果の保存方法はlogger.ipynbファイルに記載してある。  

基本的な手順は以下のとおりである。
- set_logger：出力ファイル名、出力間隔、出力内容、出力対象の設定
- write_header：出力ファイルにヘッダー名を書き込む
- write_data：出力ファイルに出力対象を書き込む

なお、出力内容には制限があるため、うまく出力がなされない場合はlogger.ipynbを参照のこと。

In [13]:
# ロガーの設定
# 全ての部屋の温湿度の結果の出力
logger_rooms = set_logger( 
            "result_case5_all_rooms",   # ファイル名
            10.0,                   # 出力間隔
            ["temp","rh","ah"],     # 出力内容
            network_model.rooms )   # 出力対象

# 壁体内の温湿度の出力
logger_walls = [ set_logger( 
            "result_case5_wall"*string(i),         # ファイル名
            10.0,                                   # 出力間隔
            ["temp","rh","ah","phi"],               # 出力内容
            network_model.walls[i].target_model )   # 出力対象
            for i = 1 : length(network_model.walls)  ]

# 各部屋の温度・湿度の形成メカニズムの分析
logger_room_analysis = set_logger( 
            "result_case5_room",   # ファイル名
            10.0,                   # 出力間隔
            ["room_analysis"],      # 出力内容
            network_model )         # 出力対象

# ヘッダーの書き込み
loggers = vcat( logger_rooms, [ logger_walls[i] for i = 1 : length(logger_walls) ], logger_room_analysis )
for files = loggers
    write_header_to_logger( files )
    write_data_to_logger( files, network_model.climate.date )
end

## 手順５：繰り返し（ループ）計算  


In [ ]:
###########################################################
# 除湿器のモデル化
# コンプレッサーの消費電力＋凝縮潜熱　＝　加熱量
# 最大除湿量　18.0L/(日) = 0.000208 kg/s
# 最大消費電力 390W
# 水の蒸発潜熱（一定値と仮定） 2250kJ/kg
# 除湿による最大発熱量 468.75W
# 実測データより：除湿器による温度上昇は5～13℃程度（平均10℃程度）
# 実測データより：除湿器による絶対湿度低下は0.0015kg/kg～0.0025kg/kg程度　⇒　風量0.1kg/sと仮定
# 風量(0.1kg/s)×比熱(1005J/kgK)×温度上昇(10K) = 1005W ≒ 最大消費電力(390W) + 最大発熱量(468.75W)なので、概ね妥当と判断

c    = 0.3 # 補正係数（除湿器の運用に合わせた係数）
H_dehumi = c * ( 390.0 + 468.75 ) * 2.0 # 除湿器は2台
J_dehumi = - c * ( 0.000208 ) * 2.0 # 除湿器は2台

# 計算回数の設定
while network_model.climate.date ≠ end_date

    # １：外界気象データの再設定
    reset_climate_data(network_model.climate)
        
    # ２：流量および収支計算
    cal_boundary_condtion_of_BNM( network_model )       # 壁面に当たる日射量・降水量の計算
    cal_network_flux_of_wall( network_model )           # 壁体を通じた熱・水分流量
    cal_network_flux_of_ventilation( network_model )    # 換気による熱・水分流量
    cal_new_value_ver_network( network_model, dt )      # 熱・水分収支計算

    # 除湿器の設定：稼働させる時期の設定（年間を通じて使う場合は1 ~ 12とする　）
    if  6 <= month(network_model.climate.date) && month(network_model.climate.date) <= 10
        set_H_in( network_model.rooms[2], H_dehumi)
        set_J_in( network_model.rooms[2], J_dehumi)
    else
        set_H_in( network_model.rooms[2], 0.0 )
        set_J_in( network_model.rooms[2], 0.0 )
    end 
    
    # ３：計算時刻の表示
    # 日本語は避けた方が無難
    if hour(network_model.climate.date) == 0 && minute(network_model.climate.date) == 0 && second(network_model.climate.date) == 0 && millisecond(network_model.climate.date) == 0;
        println(
            Dates.format(network_model.climate.date, "yyyy/mm/dd HH:MM"), 
            " 外気：温度" *  string( round(temp(network_model.climate) - 273.15, digits = 1 ) ) * "[℃]", 
            " 湿度" * string( round( rh((network_model.climate)), digits = 2 ) ) * "[-]" )
    end

    # ４：時間経過の計算
    time_elapses( network_model.climate, dt )
    
    # ５：データのロギング  
    for files = loggers
        write_data_to_logger( files, network_model.climate.date )
    end
    
end

# ファイルのクローズ
for files = loggers
    for i = 1 : length(files.file)
        close(files.file[i])
    end
end

2022/04/01 00:00 外気：温度9.4[℃] 湿度0.89[-]
2022/04/02 00:00 外気：温度6.2[℃] 湿度0.48[-]
2022/04/03 00:00 外気：温度9.1[℃] 湿度0.83[-]


In [15]:
network_model.walls[24]

Wall(24, "roof1", 1, 8, 0.0, 0.37, 190.8, 0.0, 0.0, Cell[Cell([1, 1, 1], [0.0, 0.0, 0.0], 0.002, 1.0, 1.0, 0.0, 1.0, 1.0, 283.1619946661878, -75830.88866795876, 0.0, 0.0, 0.0, 101325.0, "metal_plate", [[-5.626978298321344, 0.0, 0.0], [5.614748114296107, 0.0, 0.0]], [[4.593074342843591e-28, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]), Cell([2, 1, 1], [0.0, 0.0, 0.0], 0.002, 1.0, 1.0, 0.001, 1.0, 1.0, 283.34940138263556, -19953.51973423675, 0.0, 0.0, 0.0, 101325.0, "extruded_polystyrene", [[-5.614748114296107, 0.0, 0.0], [5.585545916684963, 0.0, 0.0]], [[-0.0, 0.0, 0.0], [1.1692165195608051e-8, 0.0, 0.0]], [[-0.0, 0.0, 0.0], [-0.0, 0.0, 0.0]]), Cell([3, 1, 1], [0.0, 0.0, 0.0], 0.002, 1.0, 1.0, 0.002, 1.0, 1.0, 283.90795295363233, -21355.622905731503, 0.0, 0.0, 0.0, 101325.0, "extruded_polystyrene", [[-5.585545916684963, 0.0, 0.0], [5.574192650339683, 0.0, 0.0]], [[-1.1692165195608051e-8, 0.0, 0.0], [1.607655755003226e-8, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]

# 松﨑さんへ  
計算エラーが出る場合は、主に壁体内の計算が上手くいかないことに起因します。  
エラーが出たときは下記のコードを実行し、出力結果にある青いtext editorをクリックし、どの場所の結果がおかしいか見てみてください。  
なお、print文中のtempをpvに変えれば水蒸気圧が、rhに変えれば相対湿度を見ることができます。

In [17]:
println(network_model.climate.date)
for i = 1 : length(network_model.walls)
    for j = 1 : length(network_model.walls[i].cell)
        println( "i = ",i , " j = ", j, " , " , rh(network_model.walls[i].cell[j]))
        #println( "i = ",i , " j = ", j, " , " , miu(network_model.walls[i].cell[j]))
    end
end

2022-04-04T00:00:00
i = 1 j = 1 , 0.8509548394401439
i = 1 j = 2 , 0.7063156165087363
i = 1 j = 3 , 0.6213398923416894
i = 1 j = 4 , 0.5747797424625918
i = 1 j = 5 , 0.5540584338819472
i = 1 j = 6 , 0.5503472012933696
i = 1 j = 7 , 0.5591047975629306
i = 1 j = 8 , 0.561415340220205
i = 1 j = 9 , 0.5614498878656982
i = 1 j = 10 , 0.561452398875385
i = 1 j = 11 , 0.5614543767210101
i = 1 j = 12 , 0.5614561521881868
i = 1 j = 13 , 0.5614578301069784
i = 1 j = 14 , 0.561459500747787
i = 1 j = 15 , 0.5614612422451867
i = 1 j = 16 , 0.5614631223407697
i = 1 j = 17 , 0.5614652002326828
i = 1 j = 18 , 0.5614675272463544
i = 1 j = 19 , 0.5614700076281687
i = 1 j = 20 , 0.5614604762552607
i = 1 j = 21 , 0.5610934440921493
i = 1 j = 22 , 0.6050423357970208
i = 2 j = 1 , 0.8404753136990765
i = 2 j = 2 , 0.5632687345807085
i = 2 j = 3 , 0.5614642972194915
i = 2 j = 4 , 0.561509538118611
i = 2 j = 5 , 0.5615166256121584
i = 2 j = 6 , 0.56152209198428
i = 2 j = 7 , 0.5615268694635788
i = 2 j = 8 , 0.

In [18]:
network_model.walls[24].target_model[2]

Cell([1, 1, 1], [0.0, 0.0, 0.0], 0.002, 1.0, 1.0, 0.0, 1.0, 1.0, 283.1619946661878, -75830.88866795876, 0.0, 0.0, 0.0, 101325.0, "metal_plate", [[-5.626978298321344, 0.0, 0.0], [5.614748114296107, 0.0, 0.0]], [[4.593074342843591e-28, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])